In [94]:
import os
import sys
import imp
import pprint

sys.path.append('/nuwa_cluster/home/jackyu/climatools/')
import climatools.misc as climamisc
imp.reload(climamisc)

from IPython.display import HTML

In [91]:
def related_fortran_files():
    dir_rrtmg = '/nuwa_cluster/home/jackyu/climate_models/cesm1_2_2/\
models/atm/cam/src/physics/rrtmg/'
    
    relative_filepaths = ['radlw.F90',
                         'ext/rrtmg_mcica/rrtmg_lw_rad.f90',
                         ]
    
    absolute_paths = [os.path.join(dir_rrtmg, filepath) for filepath in relative_filepaths]
    return absolute_paths

In [92]:
def read_subroutines_from_file(fpath):
    with open(fpath, mode = 'r', encoding = 'utf-8') as file:
        code = file.read()
    subroutines = climamisc.get_subroutine_bodies_from_Fortran(code)
    return subroutines


def print_subroutines(fpath):
    subroutines = read_subroutines_from_file(fpath)
    for subroutine in subroutines:
        print(climamisc.get_Fortran_subroutine_name(subroutine))
        for called_subroutine in sorted(list(climamisc.get_called_Fortran_subroutine_names(subroutine))):
            print('    ', called_subroutine)
        print()

        
def get_subroutines_from_file(fpath):
    subroutines = read_subroutines_from_file(fpath)
    return [climamisc.Fortran_subroutine_to_dict(subr) for subr in subroutines]


In [95]:
paths_fortran = related_fortran_files()

list_subrs = []
for fpath in paths_fortran:
    list_subrs.extend(get_subroutines_from_file(fpath))

d_subr_childs = climamisc.Fortran_subroutine_childs_dict(list_subrs)
d_subr_parents = climamisc.Fortran_subroutine_parents_dict(subr_childs_dict)
d_subr_childs_parents = climamisc.Fortran_subroutine_parents_childs_dict(childs = subr_childs_dict, 
                                       parents = subr_parents_dict)

pprint.pprint(d_subr_childs_parents)

{'inatm': {'childs': [], 'parents': ['rrtmg_lw_clirad_maxrandom', 'rrtmg_lw']},
 'rad_rrtmg_lw': {'childs': ['rrtmg_lw_clirad_maxrandom',
                             't_stopf',
                             'outfld',
                             't_startf',
                             'mcica_subcol_lw',
                             'rrtmg_lw',
                             'cloud_overlap_lw_clirad_maxrandom'],
                  'parents': []},
 'radlw_init': {'childs': [], 'parents': []},
 'rrtmg_lw': {'childs': ['inatm', 'cldprmc', 'setcoef', 'taumol', 'rtrnmc'],
              'parents': ['rad_rrtmg_lw']},
 'rrtmg_lw_clirad_maxrandom': {'childs': ['inatm',
                                          'cldprmc',
                                          'setcoef',
                                          'taumol',
                                          'rtrnmc'],
                               'parents': ['rad_rrtmg_lw']}}


Quick scan of cloud variables through these subroutines suggests that the flow is more or less the same as shortwave.  It appears that optical depth, $\tau$, is the only optical property passed along.